# Bottle Processing
Author: Andrew Reed

### Motivation:
Independent verification of the suite of physical and chemical observations provided by OOI are critical for the observations to be of use for scientifically valid investigations. Consequently, CTD casts and Niskin water samples are made during deployment and recovery of OOI platforms, vehicles, and instrumentation. The water samples are subsequently analyzed by independent labs for  comparison with the OOI telemetered and recovered data.

However, currently the water sample data routinely collected and analyzed as part of the OOI program are not available in a standardized format which maps the different chemical analyses to the physical measurements taken at bottle closure. Our aim is to make these physical and chemical analyses of collected water samples available to the end-user in a standardized format for easy comprehension and use, while maintaining the source data files. 

### Approach:
Generating a summary of the water sample analyses involves preprocessing and concatenating multiple data sources, and accurately matching samples with each other. To do this, I first preprocess the ctd casts to generate bottle (.btl) files using the SeaBird vendor software following the SOP available on Alfresco. 

Next, the bottle files are parsed using python code and the data renamed following SeaBird's naming guide. This creates a series of individual cast summary (.sum) files. These files are then loaded into pandas dataframes, appended to each other, and exported as a csv file containing all of the bottle data in a single data file.

### Data Sources/Software:

* **sbe_name_map**: This is a spreadsheet which maps the short names generated by the SeaBird SBE DataProcessing Software to the associated full names. The name mapping originates from SeaBird's SBE DataProcessing support documentation.

* **Alfresco**: The Alfresco CMS for OOI at alfresco.oceanobservatories.org is the source of the ctd hex, xmlcon, and psa files necessary for generating the bottle files needed to create the sample summary sheet.

* **SBEDataProcessing-Win32**: SeaBird vendor software for processing the raw ctd files and generating the .btl files.


In [1]:
# Import packages used in this notebook
import os, sys, re
import pandas as pd
import numpy as np

In [2]:
# Load the name mapping for the column names
# Specifiy the local directory
sbe_name_map = pd.read_excel('/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')
# sbe_name_map = pd.read_excel('C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')

In [90]:
basepath = '/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/'
array = 'Pioneer/'
cruise = 'Pioneer-11/'
water_dir = 'Water Sampling/'
ctd_dir = 'Leg_3/ctd/'

In [91]:
os.listdir(basepath+array+cruise+'Water Sampling/')#+water_dir)

['CTD_sampling_log_Pioneer_11_Leg_2.xlsx',
 'CTD_sampling_log_Pioneer_11_Leg_3.xlsx',
 'Salts and O2']

In [92]:
# Specify the local directory where the bottle (.btl) files are stored for a particular cruise
# dirpath = 'C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/ctd/'
btlpath = basepath+array+cruise+ctd_dir
summary_sheet_path = basepath+array+cruise+water_dir+'CTD_sampling_log_Pioneer_11_Leg_3.xlsx'
salts_and_o2_path = basepath+array+cruise+water_dir+'Salts and O2/AR31-C/'
nutrients_path = basepath+array+cruise+water_dir+''
chl_path = basepath+array+cruise+water_dir+''

In [93]:
# Parse the data for the start_time
def parse_header(header):
    """
    Parse the header of bottle (.btl) files to get critical information
    for the summary spreadsheet.
    
    Args:
        header - an object containing the header of the bottle file as a list of
            strings, split at the newline.
    Returns:
        hdr - a dictionary object containing the start_time, filename, latitude,
            longitude, and cruise id.
    """
    hdr = {}
    for line in header:
        if 'start_time' in line.lower():
            start_time = pd.to_datetime(re.split('= |\[',line)[1])
            hdr.update({'Start Time [UTC]':start_time.strftime('%Y-%m-%dT%H:%M:%SZ')})
        elif 'filename' in line.lower():
            hex_name = re.split('=',line)[1].strip()
            hdr.update({'Filename':hex_name})
        elif 'latitude' in line.lower():
            start_lat = re.split('=',line)[1].strip()
            hdr.update({'Start Latitude [degrees]':start_lat})
        elif 'longitude' in line.lower():
            start_lon = re.split('=',line)[1].strip()
            hdr.update({'Start Longitude [degrees]':start_lon})
        elif 'cruise id' in line.lower():
            cruise_id = re.split(':',line)[1].strip()
            hdr.update({'Cruise':cruise_id})
        else:
            pass
    
    return hdr
        

In [94]:
# Now write a function to autopopulate the bottle summary sample sheet
files = [x for x in os.listdir(btlpath) if '.btl' in x]
for filename in files:
    filepath = os.path.abspath(btlpath+filename)
    
    # Load the raw content into memory
    with open(filepath) as file:
        content = file.readlines()
    content = [x.strip() for x in content]
    
    # Now parse the file content
    header = []
    columns = []
    data = []
    for line in content:
        if line.startswith('*') or line.startswith('#'):
            header.append(line)
        else:
            try:
                float(line[0])
                data.append(line)
            except:
                columns.append(line)
    
    # Parse the header
    hdr = parse_header(header)
    
    # Parse the column identifiers
    column_dict = {}
    for line in columns:
        for i,x in enumerate(line.split()):
            try:
                column_dict[i] = column_dict[i] + ' ' + x
            except:
                column_dict.update({i:x})
    
    # Parse the bottle data based on the column header locations
    data_dict = {x:[] for x in column_dict.keys()}

    for line in data:
        if line.endswith('(avg)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            for i,x in enumerate(values):
                data_dict[i].append(x)
        elif line.endswith('(sdev)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            data_dict[1].append(values[0])
        else:
            pass
            
    data_dict[1] = [' '.join(item) for item in zip(data_dict[1][::2],data_dict[1][1::2])]
    
    # With the parsed data and column names, match up the data and column
    # based on the location
    results = {}
    for key,item in column_dict.items():
        values = data_dict[key]
        results.update({item:values})
        
    # Put the results into a dataframe
    df = pd.DataFrame.from_dict(results)
        
    # Now add the parsed info from the header files into the dataframe
    for key,item in hdr.items():
        df[key] = item
        
    # Get the cast number
    cast = filename[filename.index('.')-3:filename.index('.')]
    df['Cast'] = str(cast).zfill(3)
    
    # Generate a filename for the summary file
    outname = filename.split('.')[0] + '.sum'
    
    # Save the results
    df.to_csv(btlpath+outname)

In [95]:
# Now, for each "summary" file, load and append to each other
df = pd.DataFrame()
for file in os.listdir(btlpath):
    if '.sum' in file:
        df = df.append(pd.read_csv(btlpath+file))
    else:
        pass

In [96]:
sbe_name_map['Short Name'].apply(lambda x: str(x).lower());

In [97]:
# Rename the column title using the sbe_name_mapping 
for colname in list(df.columns.values):
    try:
        fullname = list(sbe_name_map[sbe_name_map['Short Name'].apply(lambda x: str(x).lower() == colname.lower()) == True]['Full Name'])[0]
        df.rename({colname:fullname},axis='columns',inplace=True)
    except:
        pass

In [98]:
df['Bottle Position'] = df['Bottle Position'].apply(lambda x: str( int(x) ) )
df.drop(columns='Unnamed: 0',inplace=True)
df['Cast'] = df['Cast'].apply(lambda x: str(x).zfill(3) )

In [99]:
df.to_csv(btlpath+'CTD_Summary.csv')

### Oxygen & Salinity 
Now, we need to add the 

In [100]:
def clean_sal_files(dirpath):

    # Run check if files are held in excel format or csvs
    csv_flag = any(files.endswith('.SAL') for files in os.listdir(dirpath))
    if csv_flag:
        for filename in os.listdir(dirpath):
            sample = []
            salinity = []
            if filename.endswith('.SAL'):
                with open(dirpath+filename) as file:
                    data = file.readlines()
                    for ind1,line in enumerate(data):
                        if ind1 == 0:
                            strs = data[0].replace('"','').split(',')
                            cruisename = strs[0]
                            station = strs[1]
                            cast = strs[2]
                            case = strs[8]
                        elif int(line.split()[0]) == 0:
                            pass
                        else:
                            strs = line.split()
                            sample.append(strs[0])
                            salinity.append(strs[2])
                
                    # Generate a pandas dataframe to populate data
                    data_dict = {'Cruise':cruisename,'Station':station,'Cast':cast,'Case':case,'Sample ID':sample,'Salinity [psu]':salinity}
                    df = pd.DataFrame.from_dict(data_dict)
                    df.to_csv(file.name.replace('.','')+'.csv')
            else:
                pass
    
    else:
        # If the files are already in excel spreadsheets, they've been cleaned into a
        # logical tabular format
        pass
    

def process_sal_files(dirpath):
    
    # Check if the files are excel files or not
    excel_flag = any(files.endswith('SAL.xlsx') for files in os.listdir(dirpath))
    # Initialize a dataframe for processing the salinity files
    df = pd.DataFrame()
    if excel_flag:
        for file in os.listdir(dirpath):
            if 'SAL.xlsx' in file:
                df = df.append(pd.read_excel(dirpath+file))
        df.rename({'Sample':'Sample ID','Salinity':'Salinity [psu]','Niskin #':'Niskin','Case ID':'Case'}, 
                  axis='columns',inplace=True)
        df.dropna(inplace=True)
        df['Station'] = df['Station'].apply(lambda x: str( int(x)).zfill(3))
        df['Niskin'] = df['Niskin'].apply(lambda x: str( int(x)))
        df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
    else:
        for file in os.listdir(dirpath):
            if 'SAL.csv' in file:
                df = df.append(pd.read_csv(dirpath+file))
        df.dropna(inplace=True)
        df['Station'] = df['Station'].apply(lambda x: str( int(x)).zfill(3))
        df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
        df.drop(columns=[x for x in list(df.columns.values) if 'unnamed' in x.lower()],inplace=True)

    # Save the processed summary file for salinity
    df.to_csv(dirpath+'SAL_Summary.csv')
    
    
def process_oxy_files(dirpath):
    df = pd.DataFrame()
    for filename in os.listdir(dirpath):
        if 'oxy' in filename.lower() and filename.endswith('.xlsx'):
            df = df.append(pd.read_excel(dirpath+filename)) 
            # Rename and clean up the oxygen data to be uniform across data sets
    df.rename({'Niskin #':'Niskin','Sample#':'Sample ID','Oxy':'Oxygen [mL/L]','Unit':'Units'},
              axis='columns',inplace=True)
    df.dropna(inplace=True)
    df['Station'] = df['Station'].apply(lambda x: str( int(x)).zfill(3))
    df['Niskin'] = df['Niskin'].apply(lambda x: str( int(x)))
    df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
    df['Cruise'] = df['Cruise'].apply(lambda x: x.replace('O','0'))
    
    # Save the processed summary file for oxygen
    df.to_csv(dirpath+'OXY_Summary.csv')

In [101]:
# Now process the salts and oxygen data
    # Clean the salinity
clean_sal_files(salts_and_o2_path)
    # Process the salinity files
process_sal_files(salts_and_o2_path)
    # Process the oxygen files
process_oxy_files(salts_and_o2_path)

### CTD Sampling Log

In [102]:
sample_log = pd.read_excel(summary_sheet_path,sheet_name='Summary',header=0)
sample_log.head()

,Cruise ID,Target Asset,Cast #,Date,Niskin #,Time,Target Depth,Bottom Depth [m],Trip Depth,Potential Temp,...,Ph Bottle #,DIC/TA Bottle #,Salts Bottle #,Nitrate Bottle 1,Chlorophyll Brown Bottle #,Chlorophyll Filter Sample #,Unnamed: 18,Chlorophyll Brown Bottle Volume,Chlorophyll LN Tube,Comments
0,AR31-C,CNPM,1,2018-11-02,1.0,15:21:00,NaN,133,126.0,NaN,...,1287,1288,A1,1-1,NaN,NaN,NaN,NaN,NaN,NaN
1,AR31-C,CNPM,1,2018-11-02,2.0,15:21:00,NaN,133,126.0,NaN,...,NaN,1289,A2,1-2,NaN,NaN,NaN,NaN,NaN,NaN
2,AR31-C,CNPM,1,2018-11-02,3.0,15:25:00,NaN,133,66.0,NaN,...,NaN,1290,A3,1-3,1.0,NaN,NaN,NaN,NaN,NaN
3,AR31-C,CNPM,1,2018-11-02,4.0,15:25:00,NaN,133,66.0,NaN,...,NaN,NaN,A4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AR31-C,CNPM,1,2018-11-02,5.0,15:29:00,NaN,133,27.0,NaN,...,NaN,1291,A5,1-4,2.0,NaN,NaN,NaN,NaN,NaN


### Nutrient & Chlorophyll Data

In [ ]:
nutrients = pd.read_excel(nutrients_path)
nutrients

In [ ]:
#nutrients.reset_index(inplace=True)
#nutrients.rename({'index':'Sample ID'},axis='columns',inplace=True)
#nutrients.head()

In [ ]:
chl = pd.read_excel(chl_path)
chl.head()

In [103]:
# Load the Salinity and oxygen summaries
sal = pd.read_csv(salts_and_o2_path+'SAL_Summary.csv')
if 'case' in [x.lower() for x in sal.columns.values]:
    sal['Sample ID'] = sal['Case'] + sal['Sample ID'].apply(lambda x: str(x)) 
oxy = pd.read_csv(salts_and_o2_path+'OXY_Summary.csv')
if 'case' in [x.lower() for x in oxy.columns.values]:
    oxy['Sample ID'] = oxy['Case'] + oxy['Sample ID'].apply(lambda x: str(x)) 

In [104]:
sample_log.columns.values

array(['Cruise ID', 'Target Asset', 'Cast #', 'Date', 'Niskin #', 'Time',
       'Target Depth', 'Bottom Depth [m]', 'Trip Depth', 'Potential Temp',
       'Salinity', ' Oxygen Bottle #', 'Ph Bottle #', 'DIC/TA Bottle #',
       'Salts Bottle #', 'Nitrate Bottle 1', 'Chlorophyll Brown Bottle #',
       'Chlorophyll Filter Sample #', 'Unnamed: 18',
       'Chlorophyll Brown Bottle Volume', 'Chlorophyll LN Tube',
       'Comments'], dtype=object)

In [105]:
# Now need to mak
sample_log = sample_log.merge(sal[['Station','Sample ID','Salinity [psu]']], how='left', left_on=['Cast #','Salts Bottle #'], right_on=['Station','Sample ID'])

In [106]:
sample_log.rename({'Salinity [psu]':'Discrete Salinity [psu]'},axis='columns',inplace=True)
sample_log.drop(['Station','Sample ID'],axis='columns',inplace=True)

In [107]:
sample_log.rename(columns=lambda x: x.strip(),inplace=True)

In [108]:
sample_log = sample_log.merge(oxy[['Station','Sample ID','Oxygen [mL/L]']], how='left', left_on=['Cast #','Oxygen Bottle #'], right_on=['Station','Sample ID'])

In [109]:
sample_log.rename({'Oxygen [mL/L]':'Discrete Oxygen [mL/L]'},axis='columns',inplace=True)
sample_log.drop(['Station','Sample ID'],axis='columns',inplace=True)

In [62]:
sample_log['Nitrate Bottle 1'].iloc[0]

'1-1'

In [ ]:
sample_log['Nitrate Bottle 1'] = sample_log['Nitrate Bottle 1'].apply(lambda x: x.strip() if type(x) == str else x)
sample_log['Nitrate Bottle 1'] = sample_log['Nitrate Bottle 1'].apply(lambda x: x.replace('.','') if type(x) == str else x)
sample_log = sample_log.merge(nutrients, how='left', left_on=['Nitrate Bottle 1'], right_on=['Sample ID'])

In [110]:
sample_log.head()

,Cruise ID,Target Asset,Cast #,Date,Niskin #,Time,Target Depth,Bottom Depth [m],Trip Depth,Potential Temp,...,Salts Bottle #,Nitrate Bottle 1,Chlorophyll Brown Bottle #,Chlorophyll Filter Sample #,Unnamed: 18,Chlorophyll Brown Bottle Volume,Chlorophyll LN Tube,Comments,Discrete Salinity [psu],Discrete Oxygen [mL/L]
0,AR31-C,CNPM,1,2018-11-02,1.0,15:21:00,NaN,133,126.0,NaN,...,A1,1-1,NaN,NaN,NaN,NaN,NaN,NaN,35.5091,4.004
1,AR31-C,CNPM,1,2018-11-02,2.0,15:21:00,NaN,133,126.0,NaN,...,A2,1-2,NaN,NaN,NaN,NaN,NaN,NaN,35.5127,3.932
2,AR31-C,CNPM,1,2018-11-02,3.0,15:25:00,NaN,133,66.0,NaN,...,A3,1-3,1.0,NaN,NaN,NaN,NaN,NaN,34.6183,4.528
3,AR31-C,CNPM,1,2018-11-02,4.0,15:25:00,NaN,133,66.0,NaN,...,A4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.6187,4.516
4,AR31-C,CNPM,1,2018-11-02,5.0,15:29:00,NaN,133,27.0,NaN,...,A5,1-4,2.0,NaN,NaN,NaN,NaN,NaN,32.8272,5.952


In [ ]:
sample_log.rename(columns=lambda x: x.replace('Avg:', 'Discrete'), inplace=True)
sample_log.drop(['Sample ID'],axis='columns',inplace=True)

In [ ]:
chl_df = chl[['Station-Cast #','Brown Bottle #','Chl (ug/l)','Phaeo (ug/l)']]
chl_df.rename(columns=lambda x: 'Discrete ' + x, inplace=True)
#chl_df.rename({'Discrete quality_flag':'Discrete Chl quality flag'},axis='columns',inplace=True)

In [ ]:
sample_log = sample_log.merge(chl_df, how='left', left_on=['Station-Cast #','Chlorophyll Brown Bottle #'], right_on=['Discrete Station-Cast #','Discrete Brown Bottle #'])
sample_log.drop(['Discrete Station-Cast #','Discrete Brown Bottle #'],axis='columns',inplace=True)

In [111]:
# Now load the CTD summary data
CTD = pd.read_csv(basepath+array+cruise+ctd_dir+'CTD_Summary.csv')

In [112]:
sample_log.rename({'Target Station':'Target Asset'},axis='columns',inplace=True)

In [113]:
column_list = []
for name in list(sample_log.columns.values):
    if 'Discrete' in name:
        column_list.append(name)
column_list.append('Cast #')
column_list.append('Niskin #')
column_list.append('Target Asset')
column_list.append('Bottom Depth [m]')

In [114]:
discrete_data = sample_log[column_list]

In [115]:
discrete_data

,Discrete Salinity [psu],Discrete Oxygen [mL/L],Cast #,Niskin #,Target Asset,Bottom Depth [m]
0,35.5091,4.004,1,1.0,CNPM,133
1,35.5127,3.932,1,2.0,CNPM,133
2,34.6183,4.528,1,3.0,CNPM,133
3,34.6187,4.516,1,4.0,CNPM,133
4,32.8272,5.952,1,5.0,CNPM,133
5,32.8276,5.941,1,6.0,CNPM,133
6,32.7257,6.179,1,7.0,CNPM,133
7,32.7243,6.192,1,8.0,CNPM,133
8,35.5498,4.012,2,1.0,PMCO,147
9,35.5502,3.920,2,2.0,PMCO,147


In [116]:
discrete_data['Cast #'] = discrete_data['Cast #'].apply(lambda x: str(int(x)).zfill(3))

/home/andrew/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [117]:
CTD['Cast'] = CTD['Cast'].apply(lambda x: str(x).zfill(3) )

In [118]:
CTD = CTD.merge(discrete_data, how='left', left_on=['Cast','Bottle Position'], right_on=['Cast #','Niskin #'])

In [119]:
CTD.columns.values

array(['Unnamed: 0', 'Bottle Position', 'Date Time',
       'Pressure, Digiquartz [db]', 'Depth [salt water, m]',
       'Latitude [deg]', 'Longitude [deg]', 'Temperature [ITS-90, deg C]',
       'Temperature, 2 [ITS-90, deg C]', 'Conductivity [S/m]',
       'Conductivity, 2 [S/m]', 'Salinity, Practical [PSU]',
       'Salinity, Practical, 2 [PSU]', 'Oxygen raw, SBE 43 [V]',
       'Oxygen, SBE 43 [ml/l]',
       'Oxygen Saturation, Garcia & Gordon [ml/l]',
       'Beam Attenuation, WET Labs C-Star [1/m]',
       'Beam Transmission, WET Labs C-Star [%]', 'Filename',
       'Start Latitude [degrees]', 'Start Longitude [degrees]', 'Cruise',
       'Start Time [UTC]', 'Cast', 'Discrete Salinity [psu]',
       'Discrete Oxygen [mL/L]', 'Cast #', 'Niskin #', 'Target Asset',
       'Bottom Depth [m]'], dtype=object)

In [120]:
CTD.drop(labels=['Unnamed: 0','Niskin #'],axis='columns',inplace=True)

In [121]:
CTD.rename({'Cast #':'Station'},inplace=True)

In [122]:
CTD.fillna(-9999999, inplace=True)
CTD['Cruise ID'] = sample_log['Cruise ID'][0]
CTD['Bottle Closure Time [UTC]'] = CTD['Date Time'].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m-%dT%H:%M:%SZ'))
CTD.drop(columns='Date Time', inplace=True)

In [123]:
cruise_name = 'Pioneer-11_Leg-3'
cruise_id = list(set(CTD['Cruise ID']))[0]
current_date = pd.to_datetime(pd.datetime.now()).tz_localize(tz='US/Eastern').tz_convert(tz='UTC')
version = '1-00'

In [124]:
filename = '_'.join([cruise_name,cruise_id,'Discrete','Summary',current_date.strftime('%Y-%m-%d'),'ver',version,'.xlsx'])
filename

'Pioneer-11_Leg-3_AR31-C_Discrete_Summary_2019-03-13_ver_1-00_.xlsx'

In [125]:
column_order = pd.read_excel(basepath+'column_order.xlsx')

In [126]:
column_order = tuple([x.replace('CTD','').strip() for x in column_order.columns.values])

In [127]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [128]:
results = {}
CTDsorted = pd.DataFrame()
for column in column_order:
    match = process.extractBests(column.replace('Discrete ','').replace('Calculated ',''),
                                 CTD.columns.values, limit=2, score_cutoff=56, scorer=fuzz.ratio)
    if 'flag' in column.lower():
        CTDsorted[column] = -9999999
    elif len(match) == 0:
        CTDsorted[column] = -9999999
    elif match[0][0] not in [x[0] for x in results.values()]:
        CTDsorted[match[0][0]] = CTD[match[0][0]]
        results.update({column:match[0]})
    else:
        CTDsorted[match[1][0]] = CTD[match[1][0]]
        results.update({column:match[1]})

In [129]:
match

[]

In [130]:
CTDsorted.columns.values

array(['Cruise', 'Station', 'Target Asset', 'Start Latitude [degrees]',
       'Start Longitude [degrees]', 'Start Time [UTC]', 'Cast',
       'Cast Flag', 'Bottom Depth [m]', 'Filename', 'File Flag',
       'Bottle Position', 'Niskin Flag', 'Bottle Closure Time [UTC]',
       'Pressure, Digiquartz [db]', 'Pressure Flag',
       'Depth [salt water, m]', 'Latitude [deg]', 'Longitude [deg]',
       'Temperature [ITS-90, deg C]', 'Temperature 1 Flag',
       'Temperature, 2 [ITS-90, deg C]', 'Temperature 2 Flag',
       'Conductivity [S/m]', 'Conductivity 1 Flag',
       'Conductivity, 2 [S/m]', 'Conductivity 2 Flag',
       'Salinity, Practical [PSU]', 'Salinity, Practical, 2 [PSU]',
       'Oxygen, SBE 43 [ml/l]', 'Oxygen Flag',
       'Oxygen Saturation, Garcia & Gordon [ml/l]',
       'Fluorescence [mg/m^3]', 'Fluorescence Flag',
       'Beam Attenuation, WET Labs C-Star [1/m]',
       'Beam Transmission, WET Labs C-Star [%]', 'Transmissometer Flag',
       'pH', 'pH Flag', 'Discrete 

In [132]:
CTDsorted['Discrete Oxygen [mL/L]']

0           4.004
1           3.932
2           4.528
3           4.516
4           5.952
5           5.941
6           6.179
7           6.192
8           4.012
9           3.920
10          4.392
11          4.385
12          6.062
13          6.060
14          6.139
15          6.121
16   -9999999.000
17          4.259
18          4.263
19          4.952
20          4.983
21          6.022
22          6.023
23          6.200
24          6.178
25   -9999999.000
26   -9999999.000
27   -9999999.000
28   -9999999.000
29   -9999999.000
         ...     
48          4.903
49          5.397
50          5.419
51          6.108
52          6.111
53          4.540
54          4.520
55          3.171
56          3.173
57          4.381
58          4.392
59          6.085
60          6.118
61   -9999999.000
62   -9999999.000
63   -9999999.000
64   -9999999.000
65   -9999999.000
66   -9999999.000
67   -9999999.000
68   -9999999.000
69   -9999999.000
70   -9999999.000
71   -9999999.000
72   -9999

In [133]:
# Now save the results
CTDsorted.to_excel(basepath+array+cruise+filename)

In [ ]:
CTDsorted.columns.values

In [ ]:
results